In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [2]:
file_name = "NF-CSE-CIC-IDS2018-v2.csv"
data = pd.read_csv(file_name)

In [3]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [7]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [8]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,...,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557
Bot,14310,14310,14310,14310,14310,14310,14310,14310,14310,14310,...,14310,14310,14310,14310,14310,14310,14310,14310,14310,14310
Brute Force -Web,214,214,214,214,214,214,214,214,214,214,...,214,214,214,214,214,214,214,214,214,214
Brute Force -XSS,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
DDOS attack-HOIC,108086,108086,108086,108086,108086,108086,108086,108086,108086,108086,...,108086,108086,108086,108086,108086,108086,108086,108086,108086,108086
DDOS attack-LOIC-UDP,211,211,211,211,211,211,211,211,211,211,...,211,211,211,211,211,211,211,211,211,211
DDoS attacks-LOIC-HTTP,30730,30730,30730,30730,30730,30730,30730,30730,30730,30730,...,30730,30730,30730,30730,30730,30730,30730,30730,30730,30730
DoS attacks-GoldenEye,2772,2772,2772,2772,2772,2772,2772,2772,2772,2772,...,2772,2772,2772,2772,2772,2772,2772,2772,2772,2772
DoS attacks-Hulk,43265,43265,43265,43265,43265,43265,43265,43265,43265,43265,...,43265,43265,43265,43265,43265,43265,43265,43265,43265,43265


In [9]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
10660988,172.31.66.109,172.31.0.2,1.401126e-08,1.108397e-08,0.000078,1.237078e-06,0.000098,1.237078e-06,1.405050e-08,1.405050e-08,...,0.0,0.0,0.0,1.458031e-07,1.458029e-07,1.136291e-07,1.109630e-08,0.000074,1.478535e-07,"[1.4011259652278659e-08, 1.1083974799736558e-0..."
15147714,172.31.66.68,172.31.0.2,1.245158e-08,1.008341e-08,0.000079,1.099371e-06,0.000097,1.099371e-06,1.248645e-08,1.248645e-08,...,0.0,0.0,0.0,1.295728e-07,1.295727e-07,9.858040e-08,9.861101e-09,0.000066,1.313950e-07,"[1.245157764089544e-08, 1.0083409899455166e-08..."
9587672,172.31.64.28,172.31.0.2,1.552131e-08,1.227854e-08,0.000077,1.370403e-06,0.000099,1.370403e-06,1.556478e-08,1.556478e-08,...,0.0,0.0,0.0,1.615169e-07,1.615167e-07,1.164452e-07,1.229219e-08,0.000082,1.637882e-07,"[1.5521310572476723e-08, 1.2278540225056638e-0..."
7746586,172.31.67.18,172.31.0.2,7.552795e-09,5.974837e-09,0.000045,6.668492e-07,0.000086,6.668492e-07,7.573948e-09,7.573948e-09,...,0.0,0.0,0.0,7.859541e-08,7.859535e-08,5.499144e-08,5.883835e-09,0.000040,7.970069e-08,"[7.55279530717906e-09, 5.974837018934741e-09, ..."
693958,172.31.67.31,172.31.0.2,3.657431e-09,2.893307e-09,0.000049,6.458418e-07,0.000115,6.458418e-07,3.667675e-09,3.667675e-09,...,0.0,0.0,0.0,3.805972e-08,3.805969e-08,2.821517e-08,2.896524e-09,0.000010,3.859495e-08,"[3.6574312874776476e-09, 2.8933070422099717e-0..."


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g

In [43]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h']
edge_features = train_g.edata['h']

for epoch in range(100):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)
    print(best)

    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))

tensor(1.3785, grad_fn=<AddBackward0>)
Epoch 00000 | Time(s) nan | Loss 1.3785 | ETputs(KTEPS) nan


/users/pg23/anshulsharma/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/users/pg23/anshulsharma/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


tensor(1.3745, grad_fn=<AddBackward0>)
Epoch 00001 | Time(s) nan | Loss 1.3745 | ETputs(KTEPS) nan
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00002 | Time(s) nan | Loss 1.3739 | ETputs(KTEPS) nan
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00003 | Time(s) 10.4079 | Loss 1.3774 | ETputs(KTEPS) 253.80
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00004 | Time(s) 10.3726 | Loss 1.3812 | ETputs(KTEPS) 254.67
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00005 | Time(s) 10.4908 | Loss 1.3829 | ETputs(KTEPS) 251.80
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00006 | Time(s) 10.5892 | Loss 1.3817 | ETputs(KTEPS) 249.46
tensor(1.3739, grad_fn=<AddBackward0>)
Epoch 00007 | Time(s) 10.5528 | Loss 1.3784 | ETputs(KTEPS) 250.32
tensor(1.3736, grad_fn=<AddBackward0>)
Epoch 00008 | Time(s) 10.4847 | Loss 1.3736 | ETputs(KTEPS) 251.94
tensor(1.3686, grad_fn=<AddBackward0>)
Epoch 00009 | Time(s) 10.5229 | Loss 1.3686 | ETputs(KTEPS) 251.03
tensor(1.3650, grad_fn=<AddBackward0>)
Epoch 00010 | Time(s)

In [45]:
print(dgi.state_dict())

OrderedDict([('encoder.layers.0.W_apply.weight', tensor([[-0.0355, -0.0041,  0.1557,  ...,  0.1742,  0.1231, -0.0261],
        [ 0.1564,  0.1060,  0.2002,  ...,  0.1972, -0.1495, -0.1820],
        [ 0.1513,  0.0872,  0.0805,  ...,  0.1926, -0.1676,  0.2177],
        ...,
        [-0.0523, -0.1853, -0.1190,  ..., -0.1079,  0.1321, -0.0012],
        [ 0.2157, -0.1179, -0.0950,  ...,  0.0720,  0.0948,  0.0848],
        [ 0.1668, -0.0985,  0.1407,  ..., -0.0064, -0.2001,  0.1592]])), ('encoder.layers.0.W_apply.bias', tensor([ 1.0232e-01, -8.1989e-02, -7.5615e-02,  9.3513e-02,  6.2228e-02,
        -3.4318e-02, -1.5065e-02, -9.4124e-02,  8.4912e-02,  5.7707e-02,
         8.1954e-02, -4.2385e-02, -9.4747e-02,  5.3199e-02,  4.8826e-02,
        -9.0551e-02,  1.4368e-02, -2.2409e-02, -2.4087e-02,  7.5786e-02,
        -6.4988e-02, -1.0907e-01,  4.7972e-02,  7.0384e-03, -7.6541e-02,
        -1.1493e-01,  2.7699e-02, -1.0571e-01,  1.8931e-02, -1.1075e-01,
         4.5187e-02, -9.9092e-02, -6.5918e-

In [24]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [25]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [26]:
# Convert to GPU
test_g = test_g

In [27]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [28]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [29]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.186130,0.089536,0.234762,0.499413,0.610749,-0.108356,-0.442557,-0.095262,0.068685,0.096509,...,-0.100570,0.261413,0.077442,-0.006520,0.272711,-0.072337,-0.090303,-0.199628,Benign,0
1,-0.186130,0.089536,0.234762,0.499413,0.610749,-0.108356,-0.442557,-0.095262,0.068685,0.096509,...,-0.100570,0.261413,0.077442,-0.006520,0.272711,-0.072337,-0.090303,-0.199628,Benign,0
2,-0.186130,0.089536,0.234762,0.499413,0.610749,-0.108356,-0.442557,-0.095262,0.068685,0.096509,...,-0.100570,0.261413,0.077442,-0.006520,0.272711,-0.072337,-0.090303,-0.199628,Benign,0
3,-0.186130,0.089536,0.234762,0.499413,0.610749,-0.108356,-0.442557,-0.095262,0.068685,0.096509,...,-0.100570,0.261413,0.077442,-0.006520,0.272711,-0.072337,-0.090303,-0.199628,Benign,0
4,-0.186130,0.089536,0.234762,0.499413,0.610749,-0.108356,-0.442557,-0.095262,0.068685,0.096509,...,-0.100570,0.261413,0.077442,-0.006520,0.272711,-0.072337,-0.090303,-0.199628,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641549,-0.184909,0.099202,0.243210,0.494740,0.596927,-0.109210,-0.437399,-0.103650,0.072978,0.098303,...,-0.100357,0.261067,0.081300,-0.022048,0.282673,-0.056804,-0.085026,-0.186575,Benign,0
2641550,-0.193974,0.120719,0.243462,0.516168,0.583101,-0.084616,-0.444571,-0.132814,0.085291,0.093678,...,-0.084378,0.268192,0.070342,-0.044484,0.320765,-0.016463,-0.070495,-0.216916,Benign,0
2641551,-0.191958,0.123398,0.250685,0.501447,0.574978,-0.084962,-0.455707,-0.125891,0.064002,0.079514,...,-0.083362,0.268188,0.070823,-0.048042,0.308147,-0.025993,-0.064052,-0.212066,Benign,0
2641552,-0.193751,0.122689,0.249479,0.508969,0.582178,-0.082512,-0.451045,-0.130019,0.073107,0.084857,...,-0.086211,0.268132,0.068637,-0.048151,0.312106,-0.021339,-0.071330,-0.216649,Benign,0


# Embeddings CBLOF  Embeddings

In [30]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [31]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [47]:
print(test_samples)

              0         1         2         3         4         5         6    \
0       -0.186131  0.089346  0.234351  0.499705  0.611004 -0.108287 -0.442813   
1       -0.186131  0.089346  0.234351  0.499705  0.611004 -0.108287 -0.442813   
2       -0.186131  0.089346  0.234351  0.499705  0.611004 -0.108287 -0.442813   
3       -0.186131  0.089346  0.234351  0.499705  0.611004 -0.108287 -0.442813   
4       -0.186131  0.089346  0.234351  0.499705  0.611004 -0.108287 -0.442813   
...           ...       ...       ...       ...       ...       ...       ...   
1132126 -0.184152  0.098381  0.238208  0.498316  0.594795 -0.112433 -0.433364   
1132127 -0.193824  0.122057  0.248642  0.508490  0.580410 -0.082905 -0.450611   
1132128 -0.182810  0.094784  0.238498  0.495648  0.605904 -0.114430 -0.437895   
1132129 -0.185517  0.101051  0.242561  0.496381  0.593074 -0.108055 -0.436051   
1132130 -0.183881  0.095285  0.239258  0.496412  0.599172 -0.112877 -0.434997   

              7         8  

In [32]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [33]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]/users/pg23/anshulsharma/.local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  3%|▎         | 1/36 [02:24<1:24:05, 144.16s/it]/users/pg23/anshulsharma/.local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  6%|▌         | 2/36 [04:30<1:15:36, 133.41s/it]/users/pg23/anshulsharma/.local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default

{'n_estimators': 3, 'con': 0.001}
0.9446504908193768
              precision    recall  f1-score   support

           0     0.9772    0.9987    0.9878    996643
           1     0.9882    0.8287    0.9015    135488

    accuracy                         0.9783   1132131
   macro avg     0.9827    0.9137    0.9447   1132131
weighted avg     0.9785    0.9783    0.9775   1132131



In [44]:
print(dgi.load_state_dict(torch.load('best_dgi.pkl')))

<All keys matched successfully>


In [46]:
dgi.eval()

DGI(
  (encoder): SAGE(
    (layers): ModuleList(
      (0): SAGELayer(
        (W_apply): Linear(in_features=78, out_features=128, bias=True)
        (W_edge): Linear(in_features=256, out_features=256, bias=True)
      )
    )
  )
  (discriminator): Discriminator()
  (loss): BCEWithLogitsLoss()
)

In [51]:
with torch.no_grad():
    predictions = dgi(3,0.1,test_samples[0])

AttributeError: 'int' object has no attribute 'local_scope'